# **Signa_Lab ITESO:** Generador de *Embbeddings*

## **Cuaderno 03:** Exploración y visualización de *embeddings*, consultas y relaciones semánticas (*ngramas*).
Cuaderno de código para explorar y visualizar relaciones semánticas codificadas en *embeddings* (vectores) y clústeres previamente procesados (ver  [cuaderno 01](https://github.com/signalab/generador-embeddings/blob/main/cuadernos/01_Signa_Lab_generador_embeddings_Depuraci%C3%B3n_importar_limpiar_depurar_texto_01.ipynb) y [cuaderno 02](https://github.com/signalab/generador-embeddings/blob/main/cuadernos/02_Signa_Lab_generador_embeddings_Generar_procesar_reducir_clusterizar_embeddings_01.ipynb)), con ayuda de modelos de lenguaje de la librería [sentence-transformers](https://www.sbert.net/), alojados en repositorios de [HuggingFace](https://huggingface.co/sentence-transformers), así como desde la aplicación de técnicas establecidas de procesamiento de lenguaje natural (PNL), como TF-IDF y ngramas.

**\***Los grupos de celdas marcadas con **asterisco requieren información** antes de seguir adelante.

## 1. Importar librerías, archivos de datos y modelos de lenguaje

### Instalar e importar librerías:

**Instalar librerías:**

In [ ]:
# Instalar librerías de Python necesarias

!pip install numpy
!pip install pandas
!pip install matplotlib
!pip install scikit-learn
!pip install plotly
# !pip install umap-learn
!pip install sentence_transformers

!pip install matplotlib
# !pip install yellowbrick
!pip install networkx


**Importar librerías:**

In [ ]:
# Importar librerías de Python necesarias

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# from sklearn.cluster import KMeans
import plotly.express as px
# from sklearn.decomposition import PCA
# import umap
# from sklearn.manifold import TSNE
import operator
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.collocations import BigramAssocMeasures, BigramCollocationFinder, TrigramCollocationFinder
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sentence_transformers import SentenceTransformer, util

from sklearn import datasets
import matplotlib.pyplot as plt
# from yellowbrick.cluster import SilhouetteVisualizer


from IPython.display import display, clear_output
import ipywidgets as widgets
import networkx as nx
import plotly.graph_objects as go

nltk.download('punkt')
nltk.download('stopwords')

### *Indicar rutas de archivos de datos a importar y nombre de proyecto:

**Importar (y concatenar, en caso de ser múltiples) archivos de datos** con embeddings y clústeres procesados (en formato JSON o CSV):

In [ ]:
# Definir función para cargar archivos a partir de la extensión en su ruta indicada
def load_file(path):
    if path.endswith('.csv'):
        return pd.read_csv(path)
    elif path.endswith('.xlsx'):
        return pd.read_excel(path)
    elif path.endswith('.json'):
        return pd.read_json(path)
    else:
        raise ValueError("Formato no compatible. Por favor carga solo archivos .csv or .xlsx.")

# Inicializar lista para alojar todas las rutas y una variable para el DataFrame final, accesible globalmente
file_paths = []
dfs = []
df = None  # DataFrame global

# Definir función para añadir un nuevo campo de texto (input) para añadir una ruta de archivo adicional
def add_file_input(b=None):
    path_input = widgets.Text(value='', placeholder='Escribe la ruta del archivo', description=f'Archivo {len(file_paths) + 1}:')
    file_paths.append(path_input)
    update_ui()

# Definir función para eliminar el último campo de texto (input) para ruta de archivo
def remove_file_input(b=None):
    if file_paths:
        file_paths.pop()
        update_ui()

# Definir función para procesar y cargar todos los archivos
def process_files(b):
    global dfs, df
    dfs = []  # Vaciar DataFrames

    for path_input in file_paths:
        path = path_input.value
        try:
            temp_df = load_file(path)
            temp_df['filename'] = path  # Add a column with the filename
            dfs.append(temp_df)
            print(f"Nombre de archivo: {path}")
            print(f"Filas/Columnas (shape): {temp_df.shape}")
        except ValueError as e:
            print(f"Error al cargar el archivo {path}: {e}")
            return

    if dfs:
        df = pd.concat(dfs, ignore_index=True)  # Concatenate all DataFrames
        print("\n¡Se cargaron todos los archivos!")
        print(f"Filas/Columnas (shape) de DataFrame creado: {df.shape}")

# Campo de texto (input) para indicar nombre del proyecto (para integrarse en nombres de archivos a exportar)
project_name = widgets.Text(value='', placeholder='Escribe el nombre del proyecto (corto y sin espacios)', description='Nombre del proyecto:')

# Botones para añadir y eliminar archivos
add_button = widgets.Button(description="Añadir archivo",button_style='')
remove_button = widgets.Button(description="Eliminar archivo", button_style='warning')
load_button = widgets.Button(description="Cargar archivos", button_style='primary')

add_button.on_click(add_file_input)
remove_button.on_click(remove_file_input)
load_button.on_click(process_files)

# Definir función para actualizar UI
def update_ui():
    clear_output()
    display(project_name)
    for path_input in file_paths:
        display(path_input)
    display(widgets.HBox([add_button, remove_button]))
    display(load_button)

# Inicializar UI con un campo de texto (input) para ruta de archivo
add_file_input()


### Previsualizar datos importados:

In [ ]:
# Hacer una copia de trabajo del DataFrame global con datos importados
dfClusterizado = df
print(df.info())

**Previsualizar tabla** con todos los registros importados:

In [ ]:
# Previsualizar dataframe con CSVs importados
display(dfClusterizado)

**Exportar copia en CSV con registros importados (o concatenados):**

In [ ]:
# Exportar archivo CSV con tabla de registros importados (y concatenados, en el caso de múltiples archivos)
dfClusterizado.to_csv(f"{project_name.value}_registros-clusterizados-importados.csv")
dfClusterizado.to_json(f"{project_name.value}_registros-clusterizados-importados.json")

### *Indicar e importar modelo de lenguaje:

**Importar modelo de lenguaje para búsqueda semántica:**

\* Para ejecutar consultas de búsqueda semántica, se debe cargar el mismo modelo de lenguaje que el utilizado para generar *embeddings* en los datos importados. Por default, se cargará el modelo de software libre [intfloat/multilingual-e5-large-instruct](https://huggingface.co/intfloat/multilingual-e5-large-instruct) (Wang et al, 2024), con la librería de aprendizaje profundo [sentence-transformers](https://www.sbert.net/) (Reimers & Gurevych, 2019), pero pueden utilizarse otros del [repositorio en Hugging Face](https://huggingface.co/sentence-transformers) de dicha librería o cargados localmente.

In [ ]:
# Opciones de modelos predefinidos
model_options = [
    "intfloat/multilingual-e5-large-instruct",
    "all-mpnet-base-v2",
    "all-MiniLM-L12-v2",
    "paraphrase-multilingual-mpnet-base-v2",
    "facebook-dpr-ctx_encoder-multiset-base",
    "otro (ruta de otro modelo en la nube o ruta local)"
]

# Crear dropdown para seleccionar modelo
model_dropdown = widgets.Dropdown(
    options=model_options,
    value=model_options[0],  # valor por defecto
    description="Modelos sugeridos:",
)

# Crear campo de texto para ruta personalizada (solo se mostrará si se elige 'otro')
model_path_text = widgets.Text(
    value='',
    placeholder='Escribe la ruta a otro modelo...',
    description='Ruta manual:',
    disabled=True  # Deshabilitado inicialmente
)

# Botón para cargar el modelo
load_button = widgets.Button(description="Cargar Modelo")
output = widgets.Output()

# Función para habilitar/deshabilitar el campo de texto según la selección
def on_model_select(change):
    if change['new'] == "otro (ruta de otro modelo en la nube o ruta local)":
        model_path_text.disabled = False  # Habilitar el input de ruta personalizada
    else:
        model_path_text.disabled = True  # Deshabilitar si se elige un modelo predefinido

# Función para cargar el modelo
def load_model(b):
    global embedder  # Hacer que embedder sea una variable global
    with output:
        clear_output()

        # Verificar si el modelo seleccionado es 'otro'
        if model_dropdown.value == "otro (ruta de otro modelo en la nube o ruta local)":
            model = model_path_text.value  # Tomar la ruta escrita por el usuario
        else:
            model = model_dropdown.value  # Tomar el modelo predefinido

        # Cargar el modelo usando SentenceTransformer
        try:
            embedder = SentenceTransformer(model)
            print(f"Modelo '{model}' cargado con éxito.")
        except Exception as e:
            print(f"Error al cargar el modelo: {e}")

# Conectar la selección del dropdown a la función
model_dropdown.observe(on_model_select, names='value')

# Conectar el botón de cargar a la función de carga de modelo
load_button.on_click(load_model)

# Mostrar widgets
display(model_dropdown, model_path_text, load_button, output)


## 2. Visualización de clústers y búsqueda semántica

### Visualizar *embeddings* y clústeres:

Definir función para **visualizar relaciones semánticas y clústeres en 3D:**

In [ ]:
def visualize3D(df, columnaEmbeddingsReducidas, columnaTexto, byCategory=False, columnaCluster=None):
  if columnaCluster is None:
    columnaCluster = "cluster"
  embeddingWithSelectedDimensions = df[columnaEmbeddingsReducidas].tolist()
  dfToPlot = {
        "X": [x[0] for x in embeddingWithSelectedDimensions],
        "Y": [y[1] for y in embeddingWithSelectedDimensions],
        "Z": [z[2] for z in embeddingWithSelectedDimensions],
        "cluster": df[columnaCluster].tolist(),
        "text": df[columnaTexto]
  }

  fig = px.scatter_3d(dfToPlot, x='X', y='Y', z='Z', color='cluster', hover_data=["text"],
                         labels={'X': 'Dimensión 1', 'Y': 'Dimensión 2', 'Z': 'Dimensión 3'},
                         title=f'Visualización de clústers semánticos',
                         color_discrete_sequence=px.colors.sequential.Viridis)
  fig.show()

**Ejecutar visualización de clústers 3D:**

In [ ]:
# Indicar nombre de columna con embeddings reducidos
columnaConEmbeddings = "embeddingsReducidos"
# Indicar columna de texto para etiquetas
columnaTextoClusterizado = "clean_text"
visualize3D(dfClusterizado, columnaConEmbeddings, columnaTextoClusterizado, columnaCluster="cluster")

### *Búsqueda semántica general y por clústeres:

**Identificar clústers más grandes en la muestra del tema** (mayor frecuencia de preguntas por similitud semántica):

In [ ]:
# Contar y enlistar los clústeres con mayor cantidad de registros (de mayor a menor)
cluster_freq = dfClusterizado['cluster'].value_counts(ascending=False).head(20)

# Crear una lista con los clústeres en orden descendente de cantidad de registros
cluster_list = cluster_freq.index.tolist()

# Imprimir los clústeres en el formato solicitado
for i, cluster in enumerate(cluster_list):
    print(f"cluster[{i}] - valor del {i+1}º clúster con más registros ({cluster_freq[cluster]} registros)")


In [ ]:
# Definir la función de producto punto
def dotProduct(embedding1, embedding2):
    result = sum(e1 * e2 for e1, e2 in zip(embedding1, embedding2))
    return result

# Definir la función de búsqueda semántica
def searchIntFloat(model, task, query, df, colText, colEmbedding, cluster=None):
    dfW = df.copy()
    if cluster is not None and cluster != 'Todos':
        dfW = dfW[dfW["cluster"] == cluster]

    def get_detailed_instruct(task_description: str, query: str) -> str:
        return f'Instruct: {task_description}\nQuery: {query}'

    # Preparar la consulta
    queries = [
        get_detailed_instruct(task, query)
    ]

    # Obtener el embedding de la consulta
    queryEmbeddings = model.encode(queries, convert_to_tensor=True, normalize_embeddings=True).tolist()[0]

    listOfTweetsAndSimilarity = []
    # Calcular la similitud con cada registro del DataFrame
    for index, row in dfW.iterrows():
        embeddingRow = row[colEmbedding]
        if type(embeddingRow) == str:
          # embeddingRow = [float(x) for x in embeddingRow.split(", ")]
          embeddinStr = dfClusterizado["Embedding"][index].replace("[", "").replace("]", "").split(", ")
          embeddingRow = [float(x) for x in embeddinStr]
        similarity = dotProduct(queryEmbeddings, embeddingRow)
        listOfTweetsAndSimilarity.append(similarity)

    # Agregar la columna de similitud al DataFrame original filtrado
    dfW['Similitud'] = listOfTweetsAndSimilarity

    # Reordenar las columnas para que "Similitud" sea la primera
    cols = ['Similitud'] + [col for col in dfW.columns if col != 'Similitud']
    dfW = dfW[cols]

    # Ordenar los resultados por similitud en orden descendente
    dfW = dfW.sort_values(by="Similitud", ascending=False).reset_index(drop=True)

    return dfW


# Definir la función para ejecutar la búsqueda basada en los widgets
def ejecutar_busqueda(b):
    # Limpiar el output antes de imprimir nuevos resultados
    with output:
        clear_output(wait=True)

        # Leer los valores de los widgets
        task = task_widget.value
        query = query_widget.value
        colText = colText_widget.value
        colEmbedding = colEmbedding_widget.value
        cluster = cluster_dropdown.value

        # Ejecutar la búsqueda
        global dfResultados
        dfResultados = searchIntFloat(embedder, task, query, dfClusterizado, colText, colEmbedding, cluster)

        # Obtener los primeros 100 valores de las columnas 'id', 'Similitud', y el texto de 'colText'
        top_100_rows = dfResultados[['id', 'Similitud', colText]].head(100)

        # Imprimir los valores en formato legible en la consola
        for idx, row in top_100_rows.iterrows():
            print(f"ID: {row['id']}, Similitud: {row['Similitud']:.4f}, Texto: {row[colText]}")

        # También se puede mostrar dfResultados.head() si es necesario para depuración
        # display(dfResultados.head())

# Crear widgets con mayor espacio para los textos
task_widget = widgets.Text(
    value='',
    placeholder='Explica el contexto de la tarea',
    description='Tarea:',
    layout=widgets.Layout(width='600px')
)

query_widget = widgets.Text(
    value='',
    placeholder='Escribe tu consulta',
    description='Consulta:',
    layout=widgets.Layout(width='600px')
)

colText_widget = widgets.Text(
    value='sem_text',
    placeholder='Nombre de columna de texto',
    description='Columna texto:',
    layout=widgets.Layout(width='600px')
)

colEmbedding_widget = widgets.Text(
    value='Embedding',
    placeholder='Nombre de columna de embeddings',
    description='Columna embeddings:',
    layout=widgets.Layout(width='600px')
)

cluster_dropdown = widgets.Dropdown(
    options=['Todos'] + cluster_list,  # Añadir 'Todos' para búsqueda en todo el dataframe
    value='Todos',
    description='Cluster:',
    layout=widgets.Layout(width='400px')
)

execute_button = widgets.Button(description="Ejecutar búsqueda con palabras clave o una frase")
execute_button.on_click(ejecutar_busqueda)

# Output para mostrar los resultados
output = widgets.Output()

# Mostrar widgets
display(task_widget, query_widget, colText_widget, colEmbedding_widget, cluster_dropdown, execute_button, output)

# Inicializar dfResultados como una variable global vacía
dfResultados = pd.DataFrame()


In [ ]:
# Configurar Pandas para truncar valores largos en las columnas
pd.set_option('display.max_colwidth', 50)  # Máximo de 50 caracteres por celda antes de truncar

# Mostrar las primeras filas del DataFrame
dfResultados.head()


### *Visualización de resultados de búsqueda semántica (mapa de árbol):

In [ ]:
# Variable global para almacenar el umbral de similitud
similarityThreshold = 0.85

# Función que se ejecuta cuando se presiona el botón "Ajustar"
def ajustar_threshold(b):
    global similarityThreshold
    try:
        # Leer el valor del input de texto y convertirlo a float
        new_threshold = float(threshold_input.value)
        if 0.0 <= new_threshold <= 1.0:
            similarityThreshold = new_threshold
            print(f"Umbral de similitud ajustado a: {similarityThreshold}")

            # Filtrar el DataFrame dfResultados y contar los registros
            filtered_df = dfResultados[dfResultados['Similitud'] >= similarityThreshold]
            record_count = len(filtered_df)
            print(f"Número de registros con similitud >= {similarityThreshold}: {record_count}")
        else:
            print("Por favor, ingrese un valor entre 0.0 y 1.0.")
    except ValueError:
        print("Por favor, ingrese un número válido.")

# Crear un input de texto para el umbral de similitud
threshold_input = widgets.Text(
    value='0.85',
    description='Umbral:',
    placeholder='Ingrese un valor entre 0.0 y 1.0'
)

# Crear un botón para ajustar el umbral
adjust_button = widgets.Button(
    description='Filtrar'
)

# Vincular el botón al ajuste del umbral
adjust_button.on_click(ajustar_threshold)

# Mostrar el input de texto y el botón
display(threshold_input, adjust_button)


In [ ]:
# Función para generar el treemap basado en la similitud
def generar_treemap(df, similarity_threshold):
    # Filtrar filas donde la columna 'title' y 'source/name' estén vacías o tengan valores nulos
    df_filtrado = df.dropna(subset=['videoTitle', 'channelTitle']).copy()

    # Filtrar los registros que tienen una similitud mayor o igual a similarity_threshold
    df_filtrado = df_filtrado[df_filtrado['Similitud'] >= similarity_threshold]

    # Asegurarse de que la columna 'cluster' sea tratada como cadena
    df_filtrado['cluster'] = df_filtrado['cluster'].astype(str)

    if df_filtrado.empty:
        print(f"No hay datos con similitud >= {similarity_threshold}")
        return

    # Crear el treemap utilizando 'source/name' y 'cluster' para las categorías y 'Similitud' para el tamaño
    fig = px.treemap(
        df_filtrado,
        path=['cluster', 'channelTitle', 'videoTitle'],  # Definir la estructura jerárquica
        values='Similitud',  # Utilizar la columna 'Similitud' para el tamaño de los rectángulos
        title=f'Treemap de Canal y Clústeres por Similitud (Umbral: {similarity_threshold})'
    )

    # Ajustar el formato y tamaño de las etiquetas
    fig.update_traces(
        texttemplate="%{label}",  # Mostrar etiquetas de texto con los nombres
        textfont=dict(size=18, color='white')  # Tamaño y color de la fuente de las etiquetas
    )

    # Ajustar los márgenes de la visualización
    fig.update_layout(margin=dict(t=50, l=25, r=25, b=25))

    # Mostrar el treemap interactivo
    fig.show()


In [ ]:
generar_treemap(dfResultados, similarityThreshold)

## 3. Análisis semántico por frecuencia de términos (TF-IDF) y de relaciones entre palabras (ngramas)

### Análisis semántico general (dataset completo):

Definir función para identificar **términos más frecuentes por método TF-IDF con toda la muestra:**

In [ ]:
# Definir función para calcular términos frecuentes por método TF-IDF para muestra completa
def generateTfidfForCompleteSample(df, columnaPreguntas, n=20):
  vectorizador_tfidf = TfidfVectorizer()
  tfidf_matriz = vectorizador_tfidf.fit_transform(df[columnaPreguntas])
  terminos = vectorizador_tfidf.get_feature_names_out()
  tfidf_promedio = tfidf_matriz.mean(axis=0).tolist()[0]
  tfidf_terminos = [(termino, tfidf) for termino, tfidf in zip(terminos, tfidf_promedio)]
  tfidf_terminos_importantes = sorted(tfidf_terminos, key=lambda x: x[1], reverse=True)[:n]
  return tfidf_terminos_importantes


Definir función para identificar **relaciones entre pares de palabras (bigramas) más frecuentes:**

In [ ]:
# Definir función para calcular bigramas indicando un clúster específico u omitirlo para aplicarse con la muestra completa
def calculate_bigrams(df, nCluster=None):
    # Inicializar una lista para alojar todas las palabras en columna 'pregunta'
    all_words = []

    if nCluster is not None:
      dfW = df[df["cluster"] == nCluster]
      df = pd.DataFrame(dfW["sem_text"])
    else:
      df = pd.DataFrame(df["sem_text"])

    # Correr filtro por stopwords
    stop_words = set(stopwords.words('spanish'))

    for pregunta in df['sem_text']:
        # Tokenizar textos de columna pregunta text por palabra
        tokens = word_tokenize(pregunta, language='spanish')
        # Filtrar stopwords de tokens
        filtered_words = [word for word in tokens if word.lower() not in stop_words]
        all_words.extend(filtered_words)

    # Configurar identificador de bigramas
    bigram_measures = BigramAssocMeasures()
    bigram_finder = BigramCollocationFinder.from_words(all_words)

    # Calcular peso de bigramas utilizando frecuencia
    bigrams = bigram_finder.score_ngrams(bigram_measures.raw_freq)

    # Convertir bigramas calculados y su frecuencia en dataFrame
    bigrams_df = pd.DataFrame([(src, tgt, weight) for ((src, tgt), weight) in bigrams],
                              columns=['source', 'target', 'weight'])

    bigrams_df = bigrams_df.sort_values(by='weight', ascending=False)  # Ordenar bigramas por frecuencia

    return bigrams_df  # Regresar dataFrame con lista bigramas ordenados

Definir función para identificar **relaciones entre secuencias de 3 palabras (trigramas) más frecuentes:**

In [ ]:
# Definir función para calcular trigramas indicando un clúster específico u omitirlo para aplicarse con la muestra completa
def calculate_trigrams(df, nCluster=None):
    stop_words = set(stopwords.words('spanish'))
    all_words = []

    if nCluster is not None:
      dfW = df[df["cluster"] == nCluster]
      df = pd.DataFrame(dfW["sem_text"])
    else:
      df = pd.DataFrame(df["sem_text"])

    for pregunta in df['sem_text']:  # Adjust this to match your column name
        tokens = word_tokenize(pregunta)
        filtered_words = [word for word in tokens if word not in stop_words]
        all_words.extend(filtered_words)

    bigram_measures = BigramAssocMeasures()
    trigram_finder = TrigramCollocationFinder.from_words(all_words)

    trigrams = trigram_finder.score_ngrams(bigram_measures.raw_freq)

    trigrams_df = pd.DataFrame([(src, mid, tgt, weight) for ((src, mid, tgt), weight) in trigrams],
                               columns=['source', 'middle', 'target', 'weight'])

    trigrams_df = trigrams_df.sort_values(by='weight', ascending=False)  # Ordenar bigramas por frecuencia

    return trigrams_df   # Regresar dataFrame con lista bigramas ordenados


Ejecutar función para identificar **términos más frecuentes por método TF-IDF con toda la muestra:**

In [ ]:
# Indicar número de términos más frecuentes a enlistar por método TF-IDF
nPalabrasFrecuentesDatasetCompleto = 20
tfidfDatasetCompleto = generateTfidfForCompleteSample(dfClusterizado, "sem_text", nPalabrasFrecuentesDatasetCompleto)

In [ ]:
# Enlistar términos más frecuentes en muestra completa por método TF-IDF
tfidfDatasetCompleto

In [ ]:
df_tfidfDatasetCompleto = pd.DataFrame(tfidfDatasetCompleto, columns=["términos","frecuencias"])

In [ ]:
top_tfidf_muestra = df_tfidfDatasetCompleto.head(20)

# Crear un gráfico de barras horizontal
plt.figure(figsize=(10, 5))
plt.barh(top_tfidf_muestra['términos'], top_tfidf_muestra['frecuencias'], color='skyblue')
plt.xlabel('Frecuencia')
plt.ylabel('TF-IDF')
plt.title(f'Top 20 de Términos en Muestra Completa del Tema {project_name.value}')
plt.gca().invert_yaxis()
plt.show()

Ejecutar función para identificar **relaciones entre pares de palabras (bigramas) más frecuentes en toda la muestra:**

In [ ]:
# Ejecutar cálculo de bigramas
bigrama_muestra = calculate_bigrams(dfClusterizado)

In [ ]:
bigrama_muestra

In [ ]:
top_bigrams_muestra = bigrama_muestra.head(20)

# Crear un gráfico de barras horizontal
plt.figure(figsize=(10, 5))
plt.barh(top_bigrams_muestra['source'] + ' ' + top_bigrams_muestra['target'], top_bigrams_muestra['weight'], color='skyblue')
plt.xlabel('Frecuencia')
plt.ylabel('Bigrama')
plt.title(f'Top 20 Bigramas en Muestra Completa del Tema {project_name.value}')
plt.gca().invert_yaxis()
plt.show()

In [ ]:
pip install networkx


In [ ]:
# import networkx as nx
# import plotly.graph_objects as go

def visualize_bigrams(bigrams_df, top_n=100):
    # Crear un grafo dirigido
    G = nx.DiGraph()

    # Añadir nodos y bordes al grafo con pesos
    for _, row in bigrams_df.iterrows():
        G.add_edge(row['source'], row['target'], weight=row['weight'])

    # Calcular el grado ponderado de cada nodo
    weighted_degree = dict(G.degree(weight='weight'))

    # Ordenar nodos por grado ponderado y seleccionar los top_n
    top_nodes = sorted(weighted_degree.items(), key=lambda x: x[1], reverse=True)[:top_n]
    top_nodes = set(node for node, _ in top_nodes)

    # Crear un subgrafo con los nodos seleccionados
    H = G.subgraph(top_nodes).copy()

    # Obtener posiciones de los nodos usando el layout de Fruchterman-Reingold
    pos = nx.spring_layout(H, seed=42)

    # Obtener los bordes y los pesos para visualización
    edge_trace = go.Scatter(
        x=[],
        y=[],
        line=dict(width=0.5, color='#888'),
        hoverinfo='none',
        mode='lines'
    )

    for edge in H.edges(data=True):
        x0, y0 = pos[edge[0]]
        x1, y1 = pos[edge[1]]
        edge_trace['x'] += (x0, x1, None)
        edge_trace['y'] += (y0, y1, None)

    # Obtener los nodos para visualización
    node_trace = go.Scatter(
        x=[],
        y=[],
        text=[],
        mode='markers+text',
        textposition='top center',
        marker=dict(
            size=[],  # Dejar espacio para ajustar tamaño de nodos
            color='#1f78b4',
            line=dict(width=2, color='rgb(0,0,0)')
        )
    )

    for node in H.nodes():
        x, y = pos[node]
        node_trace['x'] += (x,)
        node_trace['y'] += (y,)
        node_trace['text'] += (node,)
        # Escalar el tamaño de los nodos por el grado ponderado
        node_trace['marker']['size'] += (weighted_degree[node] * 1000,)

    # Crear la visualización interactiva con Plotly
    fig = go.Figure(data=[edge_trace, node_trace],
                    layout=go.Layout(
                        showlegend=False,
                        hovermode='closest',
                        margin=dict(b=0, l=0, r=0, t=0)
                    ))

    fig.show()


In [ ]:
# Visualizar grafo con bigramas de top 100 nodos por grado con pesos
bigrams_df = calculate_bigrams(dfClusterizado)  # Calcular bigramas
visualize_bigrams(bigrams_df)  # Visualizar bigramas

Ejecutar función para identificar **relaciones entre secuencias de 3 palabras (trigramas) más frecuentes:**

In [ ]:
# Ejecutar cálculo de trigramas
trigrama_muestra = calculate_trigrams(dfClusterizado)
trigrama_muestra

**Visualizar trigramas de toda la muestra**

In [ ]:
top_trigrams_muestra = trigrama_muestra.head(20)

# Crear un gráfico de barras horizontal
plt.figure(figsize=(10, 5))
plt.barh(top_trigrams_muestra['source'] + ' ' + top_trigrams_muestra['middle'] + ' ' + top_trigrams_muestra['target'], top_trigrams_muestra['weight'], color='skyblue')
plt.xlabel('Frecuencia')
plt.ylabel('Trigrama')
plt.title(f'Top 20 Trigramas en Muestra Completa del Tema {project_name.value}')
plt.gca().invert_yaxis()
plt.show()

### *Análisis semántico por clústeres:


In [ ]:
# Dropdown basado en lista ordenada de top clusters, con opción de elegir manualmente algún y abrir espacio para escribirlo manualmente en un input de texto, como con modelos. Una sola función de clúster ajustable, no triplicar.

Definir función para identificar **términos más frecuentes por método TF-IDF por clúster**

In [ ]:
# Definir función para calcular términos frecuentes por método TF-IDF por clúster
def funcion_tfidf_cluster(dfClusterizado, n=20, agrupar_por_region=False, n_cluster=None):
    # Inicializar el vectorizador TF-IDF
    vectorizador_tfidf = TfidfVectorizer()

    # Calcular las características TF-IDF por cluster
    tfidf_por_cluster = {}

    if agrupar_por_region:
        for region, df_region in dfClusterizado.groupby('region'):
            clusters = df_region['cluster'].unique()
            for cluster in clusters:
                if n_cluster is not None and cluster != n_cluster:
                    continue
                preguntas_cluster = df_region[df_region['cluster'] == cluster]['sem_text'].dropna()
                tfidf_matriz = vectorizador_tfidf.fit_transform(preguntas_cluster)
                terminos = vectorizador_tfidf.get_feature_names_out()
                tfidf_promedio = tfidf_matriz.mean(axis=0).tolist()[0]
                tfidf_terminos = [(termino, tfidf) for termino, tfidf in zip(terminos, tfidf_promedio)]
                tfidf_terminos_importantes = sorted(tfidf_terminos, key=lambda x: x[1], reverse=True)[:n]
                if region not in tfidf_por_cluster:
                    tfidf_por_cluster[region] = {}
                tfidf_por_cluster[region][cluster] = tfidf_terminos_importantes
    else:
        clusters = dfClusterizado['cluster'].unique()
        for cluster in clusters:
            if n_cluster is not None and cluster != n_cluster:
                continue
            preguntas_cluster = dfClusterizado[dfClusterizado['cluster'] == cluster]['sem_text'].dropna()
            tfidf_matriz = vectorizador_tfidf.fit_transform(preguntas_cluster)
            terminos = vectorizador_tfidf.get_feature_names_out()
            tfidf_promedio = tfidf_matriz.mean(axis=0).tolist()[0]
            tfidf_terminos = [(termino, tfidf) for termino, tfidf in zip(terminos, tfidf_promedio)]
            tfidf_terminos_importantes = sorted(tfidf_terminos, key=lambda x: x[1], reverse=True)[:n]
            tfidf_por_cluster[cluster] = tfidf_terminos_importantes

    # Devolver los términos TF-IDF más prominentes por cluster y región si se agrupó por región, de lo contrario, solo por cluster
    return tfidf_por_cluster

In [ ]:
# Supongamos que esta es tu lista de clústers en orden descendente
# (Ejemplo, reemplaza esto con tu lista real de clústers)
# clusters = ['Cluster1', 'Cluster2', 'Cluster3', 'Cluster4', 'Cluster5']

# Variable global para almacenar el clúster seleccionado
filteredCluster = cluster_list[0]  # Valor inicial

# Función que se ejecuta cuando se selecciona un clúster del menú desplegable
def on_cluster_change(change):
    global filteredCluster
    filteredCluster = change['new']
    print(f"Clúster seleccionado: {filteredCluster}")

# Crear un menú desplegable con los clústers
cluster_dropdown = widgets.Dropdown(
    options=cluster_list,
    value=cluster_list[0],  # Valor inicial
    description='Elegir clúster:',
    style={'description_width': 'initial'}  # Opcional: ajustar el ancho de la descripción
)

# Vincular el menú desplegable al método on_cluster_change
cluster_dropdown.observe(on_cluster_change, names='value')

# Mostrar el menú desplegable
display(cluster_dropdown)

In [ ]:
print(filteredCluster)

**Calcular términos más frecuentes por método TF-IDF para 1er clúster:**

In [ ]:
# Indicar número de clúster y cantidad de términos a enlistar con mayor puntuación tras el análisis TF-IDF
# cluster_tfid_1 = 0
n_terminos = 20

# tfidf_por_cluster_region = funcion_tfidf(dfClusterizado, n=3, agrupar_por_region=True, n_cluster=cluster_tfid)
tfidf_por_cluster = funcion_tfidf_cluster(dfClusterizado, n=n_terminos, agrupar_por_region=False, n_cluster=filteredCluster)

# Enlistar términos con mayor frecuencia en clúster calculados por análisis TF-IDF
tfidf_por_cluster = tfidf_por_cluster[filteredCluster]
tfidf_por_cluster

In [ ]:
# tfidf_por_cluster_1

In [ ]:
# # Enlistar términos con mayor frecuencia en clúster calculados por análisis TF-IDF
# tfidf_por_cluster_1 = tfidf_por_cluster_1[filteredCluster]
# tfidf_por_cluster_1

In [ ]:
df_tfidf_cluster = pd.DataFrame(tfidf_por_cluster, columns=["términos", "frecuencias"])
df_tfidf_cluster

In [ ]:
top_tfidf_cluster = df_tfidf_cluster.head(20)

# Crear un gráfico de barras horizontal
plt.figure(figsize=(10, 5))

plt.barh(df_tfidf_cluster['términos'], df_tfidf_cluster['frecuencias'], color='skyblue')
plt.xlabel('Frecuencia')
plt.ylabel('TF-IDF')
plt.title(f'Top 20 de Términos en Clúster {filteredCluster} en Muestra del Tema {project_name.value}')
plt.gca().invert_yaxis()
plt.show()

**Calcular bigrama para 1er clúster:**

In [ ]:
# Indicar número de clúster con mayor cantidad de preguntas
# n_cluster_1 = cluster_1
bigrama_cluster = calculate_bigrams(dfClusterizado, filteredCluster)
bigrama_cluster

**Visualizar bigrama de 1er cluster** con mayor cantidad de preguntas:

In [ ]:
top_bigrams_cluster = bigrama_cluster.head(20)

# Crear un gráfico de barras horizontal
plt.figure(figsize=(10, 5))
plt.barh(top_bigrams_cluster['source'] + ' ' + top_bigrams_cluster['target'], top_bigrams_cluster['weight'], color='skyblue')
plt.xlabel('Frecuencia')
plt.ylabel('Bigrama')
plt.title(f'Top 20 Bigramas en Clúster {filteredCluster} en Muestra del Tema {project_name.value}')
plt.gca().invert_yaxis()
plt.show()

In [ ]:
visualize_bigrams(bigrama_cluster)  # Visualizar bigramas

**Calcular trigrama para 1er clúster:**

In [ ]:
trigrama_cluster = calculate_trigrams(dfClusterizado, filteredCluster)
trigrama_cluster

In [ ]:
top_trigrams_cluster = trigrama_cluster.head(20)

# Crear un gráfico de barras horizontal
plt.figure(figsize=(10, 5))
plt.barh(top_trigrams_cluster['source'] + ' ' + top_trigrams_cluster['middle'] + ' ' + top_trigrams_cluster['target'], top_trigrams_cluster['weight'], color='skyblue')
plt.xlabel('Frecuencia')
plt.ylabel('Trigrama')
plt.title(f'Top 20 Trigramas en Clúster (número {filteredCluster}) en Muestra del Tema {project_name.value}')
plt.gca().invert_yaxis()
plt.show()

## 4. Extracción manual de registros por ID (opcional)

### *Previsualizar filas a extraer manualmente por ID:

In [ ]:
# Previsualizar registro por ID
dfClusterizado[dfClusterizado["id"]== 1006310]

In [ ]:
# Previsualizar registro por ID
dfClusterizado[dfClusterizado["id"]== 1005157]

### *Generar tabla con registros seleccionados por ID:

In [ ]:
# Definir función que recibe una lista de IDs para crear DataFrame con los registros a extraer
def seleccionar_preguntas_por_ids(dfClusters, dfSelectedQuestions, selectedIDs):
    # Crear una copia del DataFrame original para evitar modificaciones directas
    dfClusters_copia = dfClusters.copy()

    # Filtrar los registros seleccionados basados la lista de IDs proporcionados
    df_seleccionados = dfClusters_copia[dfClusters_copia["id"].isin(selectedIDs)]

    # Eliminar de la copia del DataFrame global los registros seleccionadoss
    dfClusters_copia = dfClusters_copia[~dfClusters_copia["id"].isin(selectedIDs)]

    # Si dfSelectedQuestions está vacío, inicializarlo con las columnas de dfClusters
    if dfSelectedQuestions.empty:
        dfSelectedQuestions = pd.DataFrame(columns=dfClusters.columns)

    # Obtener el índice para las nuevas filas en el DataFrame de registros seleccionados
    nuevo_indice = len(dfSelectedQuestions)

    # Agregar las filas seleccionadas al DataFrame de registros seleccionados
    for _, row in df_seleccionados.iterrows():
        dfSelectedQuestions.loc[nuevo_indice] = row
        nuevo_indice += 1

    return dfClusters_copia, dfSelectedQuestions

**Enlistar IDs de resgistros seleccionados** para extraer en una tabla nueva:

In [ ]:
selectedIDs = [1006310, 1005157, 1003136]  # Lista de IDs seleccionados

dfClusters = dfClusterizado.copy()
dfSelectedQuestions = pd.DataFrame(columns=dfClusters.columns)  # DataFrame vacío para registros seleccionados

dfClusters_actualizado, dfSelectedQuestions_actualizado = seleccionar_preguntas_por_ids(dfClusters, dfSelectedQuestions, selectedIDs)

In [ ]:
dfClusters_actualizado.shape

In [ ]:
dfSelectedQuestions_actualizado

In [ ]:
#Verificar eliminacion por id
id_por_verficar = '1000103'
dfClusters_actualizado[dfClusters_actualizado['id'] == id_por_verficar]

In [ ]:
# Exportar archivo de datos (en formato CSV) con registros seleccionados
dfSelectedQuestions_actualizado.to_csv(f"{project_name.value}RegistrosSeleccionados.csv")

In [ ]:
# Exportar archivo de datos (en formato CSV) con el resto de registros no seleccionados
dfClusters_actualizado.to_csv(f"{project_name.value}RegistrosSinSeleccionados.csv")

## 5. Referencias:

* Bird, Steven, Edward Loper & Ewan Klein (2009). Natural Language Processing with Python.  O'Reilly Media Inc.
* McInnes, L., Healy, J., & Melville, J. (2018). Umap: Uniform manifold approximation and projection for dimension reduction. arXiv preprint arXiv:1802.03426. https://arxiv.org/abs/1802.03426
* Pedregosa, F., Varoquaux, G., Gramfort, A., Michel, V., Thirion, B., Grisel, O., Blondel, M., Prettenhofer, P., Weiss, R., Dubourg, V., Vanderplas, J., Passos, A., Cournapeau, D., Brucher, M., Perrot, M., Duchesnay, E., & others. (2011). Scikit-learn: Machine Learning in Python. Journal of Machine Learning Research, 12, 2825–2830.
* Reimers, N., & Gurevych, I. (2019). Sentence-BERT: Sentence Embeddings using Siamese BERT-Networks (arXiv:1908.10084). arXiv. http://arxiv.org/abs/1908.10084
* Rousseeuw, P. (1987). Silhouettes: a Graphical Aid to the Interpretation and Validation of Cluster Analysis. Computational and Applied Mathematics. 20: 53–65. doi:10.1016/0377-0427(87)90125-7.
* Spärck-Jones, K. (1972). A statistical interpretation of term specificity and its application in retrieval. Journal of Documentation, 28(1), 11-21. https://www.staff.city.ac.uk/~sbrp622/idfpapers/ksj_orig.pdf
* Thorndike, R. (1953). Who Belongs in the Family?. Psychometrika. 18 (4): 267–276. doi:10.1007/BF02289263. S2CID 120467216.
* Wang, L., Yang, N., Huang, X., Yang, L., Majumder, R., & Wei, F. (2024). Multilingual E5 Text Embeddings: A Technical Report. arXiv preprint arXiv:2402.05672. Recuperado de https://arxiv.org/abs/2402.05672

*Programación asistida ocasionalmente con herramientas de IA Generativa: ChatGPT, Phind, Google Gemini y Perplexity


## 6. Créditos

**Realizado por el equipo de Signa_Lab ITESO:**

- **Programación de cuadernos de código (Python)**:
Javier de la Torre Silva, José Luis Almendarez González y Diego Arredondo Ortiz

- **Supervisión del desarrollo tecnológico y documentación:**
Diego Arredondo Ortiz

- **Equipo de Coordinación Signa_Lab ITESO:**
Paloma López Portillo Vázquez, Víctor Hugo Ábrego Molina y Eduardo G. de Quevedo Sánchez

Mayo, 2024. Instituto Tecnológico y de Estudios Superiores de Occidente (ITESO)
Tlaquepaque, Jalisco, México.
